In [2]:
import pandas as pd
import numpy as np

In [112]:
def merge_adj(heat_set):
    heat_set = sorted(heat_set, key=lambda x: x[-1])
    # print("before truncate: ", (heat_set))
    num_drop_off = int(len(heat_set) * 0.4)
    del heat_set[:num_drop_off]
    third_elements = [t[2] for t in heat_set]
    # plot_heatness_cdf(third_elements, "to_drop_off")
    # heat_set = [tuple(item[:2]) if len(item) >
    #             2 else item for item in heat_set]
    heat_set = {(x, y) for x, y, _ in heat_set}
    print("after truncate: ", (heat_set))
    while True:
        merged = set()
        for tuple1 in heat_set:
            remaining_counter = len(heat_set) - 1
            for tuple2 in heat_set:
                if tuple1 != tuple2:
                    if tuple1[1] >= tuple2[0] and tuple1[0] <= tuple2[1]:
                        merged.add(
                            (min(tuple1[0], tuple2[0]), max(tuple1[1], tuple2[1])))
                        print("adding merged: {}-{}".format(min(tuple1[0], tuple2[0]),max(tuple1[1], tuple2[1])))
                    else:
                        remaining_counter -= 1
                    print(remaining_counter)
            if remaining_counter == 0:
                print("adding standalone")
                merged.add(tuple1)
        # break
        if merged == heat_set:
            break
        else:
            print("continue merging...")
            heat_set = merged.copy()
    return heat_set

In [7]:
def _merge_adj(heat_set):
    while True:
        merged = set()
        for tuple1 in heat_set:
            remaining_counter = len(heat_set) - 1
            for tuple2 in heat_set:
                if tuple1 != tuple2:
                    if tuple1[1] >= tuple2[0] and tuple1[0] <= tuple2[1]:
                        merged.add(
                            (min(tuple1[0], tuple2[0]), max(tuple1[1], tuple2[1])))
                    else:
                        remaining_counter -= 1
            if remaining_counter == 0:
                merged.add(tuple1)
        # break
        if merged == heat_set:
            break
        else:
            heat_set = merged.copy()
    return heat_set

In [5]:
def get_heatness_thresh_chunk(filtered_sorted_df, temp_split_percent=0.8):
    largest_temp = filtered_sorted_df['temp'].iloc[-1]
    # print("largest temp:", largest_temp)
    temp_split_value = (temp_split_percent * largest_temp)
    print("temp_split_value: ", temp_split_value)
    # differences = np.abs(filtered_sorted_df['temp'] - temp_split_value)
    # closest_index = differences.idxmin()
    # heatness_split_raw_temp = filtered_sorted_df.loc[closest_index]['temp']
    mask = filtered_sorted_df['temp'] > temp_split_value
    heatness_thresh_chunk = filtered_sorted_df[mask]
    return heatness_thresh_chunk

In [6]:
def merge_heats(filtered_sorted_all, abs_all):
    heat_list = []  # a list contains merged tuples (start_addr, end_addr)
    heat_set = set()  # a set contains merged tuples (start_addr, end_addr)
    considered_heats = set()  # a list contains index that has been considered
    smallest_temp = filtered_sorted_all.iloc[0]
    merge_expand_thresh = 0.5
    merge_expand_value = smallest_temp['temp'] * merge_expand_thresh
    # print("smallest:", smallest_temp)
    print("smallest:", merge_expand_value)
    # time.sleep(10)
    for index, latent_heat_row in filtered_sorted_all.iterrows():
        heats_sum = 0
        # if current addr is considered, skip
        if index in considered_heats:
            continue
        # else, not considered, then mark current addr with curr time as considered
        considered_heats.add(index)
        cur_row_time = latent_heat_row['abs_time']
        start_index = 0
        end_index = 0
        prev_index = index
        next_index = index
        while abs_all.loc[prev_index]['temp'] >= merge_expand_value and abs_all.loc[prev_index]['abs_time'] == cur_row_time:
            considered_heats.add(prev_index)
            heats_sum += abs_all.loc[prev_index]['temp']
            prev_index -= 1
        # once get out, prev_index either < 0.1, or get into previous time range
        # if latter, set the start_index to its next
        if abs_all.loc[prev_index]['abs_time'] != cur_row_time:
            print("time underflow, updating to next time range")
            start_index = prev_index + 1
        else:  # set start_index to the first row in current time range that is not considered as hot
            start_index = prev_index

        # do the same for end_index
        while abs_all.loc[next_index]['temp'] >= merge_expand_value and abs_all.loc[next_index]['abs_time'] == cur_row_time:
            considered_heats.add(next_index)
            heats_sum += abs_all.loc[next_index]['temp']
            next_index += 1
        if abs_all.loc[next_index]['abs_time'] != cur_row_time:
            print("time overflow, updating to previous time range")
            end_index = next_index - 1
        else:
            end_index = next_index

        assert abs_all.loc[start_index]['abs_time'] == abs_all.loc[end_index][
            'abs_time'], "Timestamp of start_index and end_index must be matched!"
        if end_index == start_index:
            # standalone heatness, append prev and next
            assert end_index == index, "Logic bug!"
            # abs_all.loc[end_index]['abs_time'], -> for later
            cur_hot_range = (int(abs_all.loc[end_index - 1]['abs_addr']), int(
                abs_all.loc[end_index + 1]['abs_addr']), heats_sum)
        else:
            assert start_index < end_index, "Start index must be smaller than end index!"

            cur_hot_range = (int(abs_all.loc[start_index]['abs_addr']), int(
                abs_all.loc[end_index]['abs_addr']), heats_sum)
        heat_set.add(cur_hot_range)

    return heat_set

In [8]:
# workload_name = sys.argv[1]
# thresh = float(sys.argv[2])
workload_name = "gapbs_bfs_twitter_whole"
thresh = 0.1
abs_all = pd.read_csv('/home/cc/functions/run_bench/playground/{}/abs_addr_time.txt'.format(
        workload_name), sep='\t', names=['abs_time', 'abs_addr', 'temp'])
sorted_abs_all = abs_all.sort_values(by='temp')
# print(sorted_abs_all)
filtered_sorted_all = sorted_abs_all[sorted_abs_all['temp'] != 0.0]

# plot_heatness_cdf(filtered_sorted_all['temp'], "heat_cdf")
# plot_heatness_dist(filtered_sorted_all['temp'])
# chunks = get_cdf_thresh_chunk(filtered_sorted_all, 0.5)
# default: 1 - 0.1 = 90% of the hotest area will be put to DRAM
chunks = get_heatness_thresh_chunk(filtered_sorted_all, thresh)
print(chunks)
if len(chunks) == 0:
    print("no hot regions observed")
    exit(0)

# # heat_list: [(time_start, time_end, addr_start, addr_end, heat_weight)]


temp_split_value:  10.0
                abs_time        abs_addr       temp
4100930  287297671213350  93824999541224  10.004663
4100931  287297671213350  93824999549084  10.004663
4100932  287297671213350  93824999556944  10.004663
4100933  287297671213350  93824999564804  10.004663
4100934  287297671213350  93824999572664  10.004663
...                  ...             ...        ...
4159598  287300338703733  93825004791704  20.000000
4159599  287300338703733  93825004799564  20.000000
4159600  287300338703733  93825004807424  20.000000
4159602  287300338703733  93825004823144  20.000000
4159997  287300338703733  93825007927844  20.000000

[341665 rows x 3 columns]


In [9]:
heat_set = merge_heats(chunks, abs_all)

smallest: 5.0023314453314
time overflow, updating to previous time range
time overflow, updating to previous time range
time underflow, updating to next time range
time overflow, updating to previous time range
time underflow, updating to next time range
time overflow, updating to previous time range
time underflow, updating to next time range
time overflow, updating to previous time range
time underflow, updating to next time range
time overflow, updating to previous time range
time underflow, updating to next time range
time overflow, updating to previous time range
time underflow, updating to next time range
time overflow, updating to previous time range
time underflow, updating to next time range
time overflow, updating to previous time range
time underflow, updating to next time range
time overflow, updating to previous time range
time underflow, updating to next time range
time overflow, updating to previous time range
time underflow, updating to next time range
time overflow, up

In [36]:
merged_res = merge_adj(heat_set)

after truncate:  {(93824992231424, 93825007943564)}
adding standalone


In [1]:
import sys
import pandas as pd

workload_name = "gapbs_pr_twitter_whole"
abs_all = pd.read_csv('/home/cc/functions/run_bench/playground/{}/abs_addr_time.txt'.format(
    workload_name), sep='\t', names=['abs_time', 'abs_addr', 'temp'])
abs_all_not_zero = abs_all[abs_all['temp'] != 0.0]

In [2]:
grouped_df = abs_all_not_zero.groupby(abs_all_not_zero.iloc[:, 1])
heat_sum = []
# add_index = 0
for addr, group_data in grouped_df:
    # print("addr group:", addr)
    # print(group_data.shape[0])
    cur_addr_heat_sum = group_data.iloc[:, 2].sum()
    heat_sum.append((addr, cur_addr_heat_sum))
    # break
heat_sum_sort_by_heat = sorted(heat_sum, key=lambda x: x[1], reverse=True)
heat_sum_sort_by_addr = sorted(heat_sum, key=lambda x: x[0], reverse=False)
print(heat_sum_sort_by_heat)
print(heat_sum_sort_by_addr)
print(len(heat_sum_sort_by_addr))

[(140735634036210, 18795.00650375252), (140735635000492, 18795.00650375252), (140735633071928, 18778.959384981237), (140735635964774, 18774.488019647695), (140735636929056, 18764.36269383038), (140735637893338, 18764.36269383038), (140735638857620, 18764.36269383038), (140735639821902, 18764.36269383038), (140735640786184, 18764.36269383038), (140735641750466, 18764.36269383038), (140735642714748, 18764.36269383038), (140735656214696, 18764.36269383038), (140735657178978, 18764.36269383038), (140735658143260, 18764.36269383038), (140735659107542, 18764.36269383038), (140735660071824, 18764.36269383038), (140735661036106, 18764.36269383038), (140735662000388, 18764.36269383038), (140735669714644, 18764.36269383038), (140735670678926, 18764.36269383038), (140735671643208, 18764.36269383038), (140735672607490, 18764.36269383038), (140735673571772, 18764.355562829365), (140735617643416, 18764.285368658406), (140735618607698, 18764.285368658406), (140735680321746, 18764.285368658406), (1407

In [11]:
split_value = 0.5  # 90% hotest regions will be preserved
heat_sum_sort_by_heat_hotest = heat_sum_sort_by_heat[0:int(len(heat_sum_sort_by_heat)*(1 - split_value))]
largest_heat = split_value * heat_sum_sort_by_heat[0][1]
print(largest_heat)
heat_sum_sort_by_heat_hotest_2 = [item for item in heat_sum_sort_by_heat if item[1] > largest_heat]

merge_expand_thresh = 0.5
merge_expand_value = heat_sum_sort_by_heat_hotest[-1][1] * merge_expand_thresh
merge_expand_value_2 = heat_sum_sort_by_heat_hotest_2[-1][1] * merge_expand_thresh
print(len(heat_sum_sort_by_heat_hotest))
print(len(heat_sum_sort_by_heat_hotest_2))
print(merge_expand_value)
print(merge_expand_value_2)
# for the dict, {addr, index}, where addr is in ascending order, including all none-zero-heat addresses
# heat_sum_dict = {}
# for i, item in enumerate(heat_sum_sort_by_addr):
#     heat_sum_dict[item] = i
# print(heat_sum_dict)
# print(heat_sum_sort_by_addr)

9397.50325187626
2292
255
24.804151986369693
5729.215479240869


: 

In [12]:
print(heat_sum_sort_by_heat_hotest[:10])
print(len(heat_sum_sort_by_heat_hotest))

[(140735634036210, 18795.00650375252), (140735635000492, 18795.00650375252), (140735633071928, 18778.959384981237), (140735635964774, 18774.488019647695), (140735636929056, 18764.36269383038), (140735637893338, 18764.36269383038), (140735638857620, 18764.36269383038), (140735639821902, 18764.36269383038), (140735640786184, 18764.36269383038), (140735641750466, 18764.36269383038)]
4126


In [13]:
considered_heats = set()
heat_set = set()
addresses_asc = [item[0] for item in heat_sum_sort_by_addr]
for each_hot_region in heat_sum_sort_by_heat_hotest:
    each_hot_region_addr = each_hot_region[0]
    if each_hot_region_addr in considered_heats:
        continue
    considered_heats.add(each_hot_region_addr)
    # cur_hot_addr_index = heat_sum_dict[each_hot_region_addr]
    cur_hot_addr_index = addresses_asc.index(each_hot_region_addr)
    # print(cur_hot_addr_index)
    start_index = 0
    end_index = 0
    prev_index = cur_hot_addr_index
    next_index = cur_hot_addr_index
    prev_addr_diff = 0
    prev_addr_diff = heat_sum_sort_by_addr[cur_hot_addr_index - 1][0] - heat_sum_sort_by_addr[cur_hot_addr_index][0]
    prev_addr_diff_2 = heat_sum_sort_by_addr[cur_hot_addr_index - 2][0] - heat_sum_sort_by_addr[cur_hot_addr_index - 1][0]
    next_addr_diff = heat_sum_sort_by_addr[cur_hot_addr_index + 1][0] - heat_sum_sort_by_addr[cur_hot_addr_index][0]
    next_addr_diff_2 = heat_sum_sort_by_addr[cur_hot_addr_index + 2][0] - heat_sum_sort_by_addr[cur_hot_addr_index + 1][0]
    if next_addr_diff == next_addr_diff_2 and prev_addr_diff != prev_addr_diff_2:
        # print("1st situ")
        # reaching the boundary of start of region, need to checking forward
        start_index = cur_hot_addr_index
        correct_addr_diff = next_addr_diff
        while heat_sum_sort_by_addr[next_index][1] >= merge_expand_value:
            considered_heats.add(next_index)
            if heat_sum_sort_by_addr[next_index + 1][0] - heat_sum_sort_by_addr[next_index][0] != correct_addr_diff:
                break
            next_index += 1
        # print("falling forward...", next_index)
        end_index = next_index
    elif next_addr_diff != next_addr_diff_2 and prev_addr_diff == prev_addr_diff_2:
        # print("2nd situ")
        # reaching ... end of region, need to check backward
        end_index = cur_hot_addr_index
        correct_addr_diff = prev_addr_diff
        while heat_sum_sort_by_addr[prev_index][1] >= merge_expand_value:
            considered_heats.add(next_index)
            if heat_sum_sort_by_addr[prev_index - 1][0] - heat_sum_sort_by_addr[prev_index][0] != correct_addr_diff:
                break
            prev_index -= 1
        # print("falling forward...", next_index)
        start_index = prev_index
    else:
        # print("3rd situ")
        while heat_sum_sort_by_addr[next_index][1] >= merge_expand_value:
            considered_heats.add(next_index)
            if heat_sum_sort_by_addr[next_index + 1][0] - heat_sum_sort_by_addr[next_index][0] != next_addr_diff:
                break
            next_index += 1
        end_index = next_index
        while heat_sum_sort_by_addr[prev_index][1] >= merge_expand_value:
            considered_heats.add(next_index)
            if heat_sum_sort_by_addr[prev_index - 1][0] - heat_sum_sort_by_addr[prev_index][0] != prev_addr_diff:
                break
            prev_index -= 1
        # print("falling forward...", next_index)
        start_index = prev_index
   
    if end_index == start_index: # standalone heat value, append prev and next
        assert end_index == cur_hot_addr_index, "Logic bug!"
        # print("start_index:", start_index)
        # print("cur_hot_addr_index:", cur_hot_addr_index)
        # print("end_index:", end_index)
        cur_hot_range = (int(addresses_asc[cur_hot_addr_index - 1]), int(addresses_asc[cur_hot_addr_index + 1]))
    else:
        assert start_index < end_index, "Start index must be smaller than end index!"
        cur_hot_range = (int(addresses_asc[start_index]), int(addresses_asc[end_index]))
    heat_set.add(cur_hot_range)
    # break
print(len(heat_set))
print(heat_set)

24
{(93824992231424, 93824992467306), (140736330247814, 140736332176378), (140736306140764, 140736308069328), (140737094923440, 140737343708196), (140737345636760, 140737354315298), (140736305176482, 140736307105046), (140736329283532, 140736331212096), (140735559786496, 140736142212824), (140736338926352, 140736340854916), (140735560750778, 140736142212824), (140736337962070, 140736339890634), (140737345636760, 140737353351016), (140737346601042, 140737354315298), (140723456516096, 140729330341096), (93824992231542, 93824992467306), (140736354354864, 140736849031530), (140736322533558, 140736324462122), (140736143177106, 140736297462226), (140736321569276, 140736323497840), (93824992231424, 93824992467188), (140723456516096, 140729324467271), (140736355319146, 140736849031530), (140723462389921, 140729330341096), (140736329283532, 140736332176378)}


In [15]:
PAGE_SIZE = 4096
def align_2_page(heat_set):
    heat_list = [list(t) for t in heat_set]
    for each_range in heat_list:
        start = each_range[0]
        end = each_range[1]
        remainder_start = start % PAGE_SIZE
        remainder_end = end % PAGE_SIZE
        if remainder_start <= 2048:
            start -= remainder_start
            # print("start:", start)
        else:
            start += PAGE_SIZE - remainder_start
            # print("start:", start)
        if remainder_end <= 2048:
            end -= remainder_end
            # print("end:", end)
        else:
            end += PAGE_SIZE - remainder_end
            # print("end:", end)
        each_range[0] = start
        each_range[1] = end
    print("modified:", heat_list)
    return heat_list

In [17]:
merged_set = _merge_adj(heat_set)
aligned_heats = align_2_page(merged_set)

modified: [[93824992231424, 93824992468992], [140736337960960, 140736340856832], [140736354353152, 140736849031168], [140737094922240, 140737343709184], [140737345638400, 140737354313728], [140736321568768, 140736324460544], [140736143175680, 140736297463808], [140736305176576, 140736308068352], [140735559786496, 140736142213120], [140736329281536, 140736332177408], [140723456516096, 140729330339840]]


: 

In [140]:
for each in merged_set:
    start_index = addresses_asc.index(each[0])
    end_index = addresses_asc.index(each[1])
    print("heatstart:",heat_sum_sort_by_addr[start_index], "heat_end:", heat_sum_sort_by_addr[end_index])
    # print()

heatstart: (93824992231424, 931.2417610593388) heat_end: (93824992467306, 931.2417610593388)
heatstart: (140736681246462, 17.600137170342062) heat_end: (140736687996436, 18.99477704341606)
heatstart: (140736555889802, 17.55634182016558) heat_end: (140736680282180, 17.600137170342062)
heatstart: (140736354354864, 23.00533962603831) heat_end: (140736456568756, 19.391011036158094)
heatstart: (140736481640088, 15.962356339214056) heat_end: (140736535639880, 17.61785553588879)
heatstart: (140737094923440, 15.225447136641957) heat_end: (140737343708196, 2.2388695609343725)
heatstart: (140736689925000, 16.906881102335106) heat_end: (140736817210224, 19.364902269706736)
heatstart: (140737345636760, 4.369514049529177) heat_end: (140737354315298, 45.62796948854498)
heatstart: (140736321569276, 25.54424671648972) heat_end: (140736324462122, 14.688289873513222)
heatstart: (140736143177106, 14.503824981957857) heat_end: (140736297462226, 6.446592913241745)
heatstart: (140735559786496, 11458.4309584

In [141]:
def write_res(hot_regions, file_path):
    with open(file_path, 'w') as file:
        for tuple_data in hot_regions:
            line = ','.join(str(element) for element in tuple_data)
            file.write(line + '\n')

In [142]:
write_res(aligned_heats, "/home/cc/res.txt")

: 